In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.2. Sociodemographic Imputation/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.2. Sociodemographic Imputation/Output')
cur_date = "121922"

library(readxl)
library(tidyverse)
library(reshape2)
library(missForest)

# reading in files
acs_df = data.frame(read_excel("ACS_Data_121222.xlsx", sheet = 2))
# need to make some of the cols numeric
acs_df[,4:30] = apply(acs_df[,4:30], 2, as.numeric)
wildfire_hazard_df = data.frame(read_excel("Wildfire_Hazard_121222.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coe

In [2]:
head(acs_df)
head(wildfire_hazard_df)

,GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,Race_Black,⋯,Poverty_Adult,Poverty_Elderly,Poverty_Male,Poverty_Female,Poverty_Black,Poverty_Asian,Poverty_Other_Race,Poverty_Mixed_Race,Poverty_Hispanic_Latino,Poverty_White
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,201.00,Alamance,87.2,37.3,21.9,78.1,15.1,61.0,28.5,⋯,20.8,14.6,26.4,28.0,54.8,0,45.3,0.0,31.3,13.9
2,14000US37001020200,202.00,Alamance,78.3,31.0,29.1,70.9,11.2,51.9,29.2,⋯,33.0,19.2,27.3,37.8,44.6,0,0.0,82.7,16.2,39.0
3,14000US37001020300,203.00,Alamance,77.8,40.7,27.1,72.9,18.9,45.2,28.9,⋯,17.3,22.0,18.0,28.3,21.0,NA,10.9,36.4,26.7,20.1
4,14000US37001020400,204.00,Alamance,78.1,30.5,31.4,68.6,10.1,19.6,59.0,⋯,26.8,14.0,30.0,33.3,19.9,NA,69.0,100.0,67.2,26.6
5,14000US37001020501,205.01,Alamance,92.7,35.1,23.3,76.7,15.6,76.1,13.3,⋯,12.2,7.1,12.3,14.8,18.9,0,0.0,53.1,25.7,10.0
6,14000US37001020502,205.02,Alamance,99.0,32.1,30.0,70.0,8.9,40.7,35.7,⋯,22.3,19.2,17.7,31.0,27.8,100,14.2,19.5,14.7,25.0


,GEO_ID,Census_Tract,County,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,14000US37001020100,201.00,Alamance,278233.79700000002,92.237034639041696,1,0-20%
2,14000US37001020200,202.00,Alamance,278233.79700000002,92.237034639041696,1,0-20%
3,14000US37001020301,203.01,Alamance,278233.79700000002,92.237034639041696,1,0-20%
4,14000US37001020302,203.02,Alamance,278233.79700000002,92.237034639041696,1,0-20%
5,14000US37001020400,204.00,Alamance,278233.79700000002,92.237034639041696,1,0-20%
6,14000US37001020501,205.01,Alamance,278233.79700000002,92.237034639041696,1,0-20%


In [3]:
# seeing if the census tracts are consistent each file
length(unique(acs_df$Census_Tract))
length(unique(wildfire_hazard_df$Census_Tract))

[1] 1410

[1] 1776

In [4]:
# they're not so what tracts aren't consistent?
`%notin%` <- Negate(`%in%`)
wildfire_hazard_df %>%
    filter(Census_Tract %notin% unique(acs_df$Census_Tract)) %>%
    select(c("GEO_ID", "Census_Tract", "County")) 

GEO_ID,Census_Tract,County
<chr>,<dbl>,<chr>
14000US37007920302,9203.02,Anson
14000US37007980000,9800.00,Anson
14000US37019020107,201.07,Brunswick
14000US37019020108,201.08,Brunswick
14000US37019020205,202.05,Brunswick
14000US37019020206,202.06,Brunswick
14000US37019020406,204.06,Brunswick
14000US37019020407,204.07,Brunswick
14000US37019020408,204.08,Brunswick


In [5]:
# lot's of missing data
dim(full_join(acs_df, wildfire_hazard_df))
dim(inner_join(acs_df, wildfire_hazard_df))

Joining, by = c("GEO_ID", "Census_Tract", "County")


[1] 3137   34

Joining, by = c("GEO_ID", "Census_Tract", "County")


[1] 1730   34

In [6]:
acs_whp_df = inner_join(acs_df, wildfire_hazard_df)

# need to make some of the cols numeric
acs_whp_df[,30:33] = apply(acs_whp_df[,30:33], 2, as.numeric)
head(acs_whp_df)

Joining, by = c("GEO_ID", "Census_Tract", "County")
Warning message in apply(acs_whp_df[, 30:33], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_whp_df[, 30:33], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_whp_df[, 30:33], 2, as.numeric):
“NAs introduced by coercion”


,GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,Race_Black,⋯,Poverty_Black,Poverty_Asian,Poverty_Other_Race,Poverty_Mixed_Race,Poverty_Hispanic_Latino,Poverty_White,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,14000US37001020100,201.00,Alamance,87.2,37.3,21.9,78.1,15.1,61.0,28.5,⋯,54.8,0,45.3,0.0,31.3,13.9,278233.8,92.23703,1,0-20%
2,14000US37001020200,202.00,Alamance,78.3,31.0,29.1,70.9,11.2,51.9,29.2,⋯,44.6,0,0.0,82.7,16.2,39.0,278233.8,92.23703,1,0-20%
3,14000US37001020400,204.00,Alamance,78.1,30.5,31.4,68.6,10.1,19.6,59.0,⋯,19.9,NA,69.0,100.0,67.2,26.6,278233.8,92.23703,1,0-20%
4,14000US37001020501,205.01,Alamance,92.7,35.1,23.3,76.7,15.6,76.1,13.3,⋯,18.9,0,0.0,53.1,25.7,10.0,278233.8,92.23703,1,0-20%
5,14000US37001020502,205.02,Alamance,99.0,32.1,30.0,70.0,8.9,40.7,35.7,⋯,27.8,100,14.2,19.5,14.7,25.0,278233.8,92.23703,1,0-20%
6,14000US37001020601,206.01,Alamance,97.3,51.7,17.5,82.5,27.8,96.9,0.9,⋯,0.0,0,100.0,0.0,6.7,3.6,278233.8,92.23703,1,0-20%


# Random Forest Imputation
Generating missing values using random forest (RF).

In [7]:
# imputing sociodemographic data using RF
random_forest_imputation = function(dataset){
    imputed_RF_object = missForest(as.matrix(dataset[,4:30]))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_final_df = data.frame(cbind(dataset[,1:3], imputed_RF_df, dataset[,31:34])) 
    
    return(imputed_final_df)
}

# calling function
imputed_df = random_forest_imputation(acs_whp_df)
head(imputed_df)

,GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,Race_Black,⋯,Poverty_Black,Poverty_Asian,Poverty_Other_Race,Poverty_Mixed_Race,Poverty_Hispanic_Latino,Poverty_White,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,14000US37001020100,201.00,Alamance,87.2,37.3,21.9,78.1,15.1,61.0,28.5,⋯,54.8,0.000,45.3,0.0,31.3,13.9,278233.8,92.23703,1,0-20%
2,14000US37001020200,202.00,Alamance,78.3,31.0,29.1,70.9,11.2,51.9,29.2,⋯,44.6,0.000,0.0,82.7,16.2,39.0,278233.8,92.23703,1,0-20%
3,14000US37001020400,204.00,Alamance,78.1,30.5,31.4,68.6,10.1,19.6,59.0,⋯,19.9,27.637,69.0,100.0,67.2,26.6,278233.8,92.23703,1,0-20%
4,14000US37001020501,205.01,Alamance,92.7,35.1,23.3,76.7,15.6,76.1,13.3,⋯,18.9,0.000,0.0,53.1,25.7,10.0,278233.8,92.23703,1,0-20%
5,14000US37001020502,205.02,Alamance,99.0,32.1,30.0,70.0,8.9,40.7,35.7,⋯,27.8,100.000,14.2,19.5,14.7,25.0,278233.8,92.23703,1,0-20%
6,14000US37001020601,206.01,Alamance,97.3,51.7,17.5,82.5,27.8,96.9,0.9,⋯,0.0,0.000,100.0,0.0,6.7,3.6,278233.8,92.23703,1,0-20%


In [8]:
# exporting data
write.csv(imputed_df, paste0(Output,"/", cur_date, "_Imputed_ACS_Data.csv"), row.names = FALSE)